# SSScoring interactive quickstart

SSScoring provides analsysis tools for individual or bulk processing of FlySight GPS competition data gathered during speed skydiving training and competition.  Scoring methodology adheres to International Skydiving Commission (ISC), International Speed Skydiving Association (ISSA), and United States Parachute Association (USPA) published competition and scoring rules.  SSScoring libraries and tools also operate with track data files produced by these devices:

- FlySight 1
- FlySight 2
- SkyTrax GPS and barometric device

SSScoring leverages data manipulation tools in the pandas and NumPy data analysis libraries.  All the SSScoring code is written in pure Python, but the implementation leverages libraries that may require native code for GPU and AI chipset support like Nvidia and M-chipsets.


---
## Requirements

- A data lake containing track data files
- Some prior Python programming to make changes to this live document or to write new implementations
- Knowledge of pandas and NumPy is a nice-to-have, but not a show-stopper

To start let's create a data lake and copy one or more of these:

- FlySight version 1 files in the form `HH-mm-ss.CSV`
- FlySight version 2 data container in the form `YY-MM-dd/HH-mm-ss/*CSV`

The `./quickstart` directory will be our data lake.  It could contain zero files or everything on a whole disk.

Check if this prerequisite is fulfilled:

In [ ]:
!mkdir -p "./quickstart-example" && if [[ -d "./quickstart-example" ]]; then ls -Al "./quickstart-example"; fi

It doesn't matter where the data lake is located.  It could be on the local device, the FlySight device itself (though reading files from it is cumbersome and super slow), a cloud drive, Amazon S3, or anywhere the system may access.  The quickstart data lake already contains a mix of FlySight 1 and 2 files.  Feel free to replace them with your files.

---
## Use case:  scoring all the jumps of a competition in 12 lines of code or less

As a speed skydiving judge, I have a directory containing a competitor's FlySight files.  I want to get the score for each jump, the aggregate score, the mean speed, and the maximum speed.  It was a short competition consisting of only 3 jumps.

### Imports constants, and getting all tracks

The previous command shows that our data lake contains files of various types, including CSVs.  SSScoring can scan a directory and its subdirectories and returns a list of all valid speed skydiving files.

In [ ]:
from ssscoring.flysight import getAllSpeedJumpFilesFrom

import pathlib

DATA_LAKE = pathlib.Path('quickstart-example/')

allFiles = getAllSpeedJumpFilesFrom(DATA_LAKE)

print('\n'.join((f.as_posix() for f in allFiles.keys())))

`allFiles` is an ordered dictionary in which each entry has these attributes set:

- a `libpath.Path` object associated with the track file
- a FlySight version tag set to 1 or 2

In [ ]:
display(allFiles)

`allFiles` includes what looks like a valid FlySight file - the one with the lower case `.csv` extension.  Compatible third-party devices like SkyTrax have a different output format and use a lower case file extension.  That's okay.  SSScoring will process their output as long as the track has the correct data headers and the file extension is CSV or csv.

### Get all the results

This is super easy--barely an inconvenience.  SSScoring sets AGL == MSL if the caller omits the optional `altitudeDZMeters` argument:

In [ ]:
from ssscoring.calc import processAllJumpFiles
import warnings
warnings.filterwarnings('ignore', category=UserWarning) # FNV, conda issue
jumpResults = processAllJumpFiles(allFiles, altitudeDZMeters=616)

The results are stored in a dictionary that uses a tagged track name as the key, and a **<a href='https://pr3d4t0r.github.io/SSScoring/ssscoring/datatypes.html#JumpResults' target='_blank'>JumpResult object</a>**:

Results keys:

In [ ]:
print('\n'.join((r for r in jumpResults.keys())))

Viewing a specific result can be overwhelming.  This use case is about reporting the results of the competition with as little fuss and as fast as possible:

In [ ]:
from ssscoring.calc import aggregateResults
from ssscoring.calc import roundedAggregateResults

summary = aggregateResults(jumpResults)
summary

Results can also be rounded, for convenience:

In [ ]:
roundedAggregateResults(summary)

And of course, we can resolve final results from the summary table:

In [ ]:
from ssscoring.calc import totalResultsFrom
totalResultsFrom(summary)

### Putting it all together

There was a lot of information to unpack for this use case because the software is doing a lot of work.  In everyday use, however, usage is quite simple.  A complete scoring system can be developed in 10 lines of code:

In [ ]:
from ssscoring.calc import aggregateResults
from ssscoring.calc import processAllJumpFiles
from ssscoring.calc import roundedAggregateResults
from ssscoring.flysight import getAllSpeedJumpFilesFrom

import pathlib

DATA_LAKE = pathlib.Path('quickstart-example/')
allFiles = getAllSpeedJumpFilesFrom(DATA_LAKE)
jumpResults = processAllJumpFiles(allFiles, altitudeDZMeters = 616)
display(aggregateResults(jumpResults))
display(totalResultsFrom(aggregateResults(jumpResults)))

If we wanted to get technical the whole implementation can be done in fewer than 5 lines of code.  The import statements do nothing other than make the functions available for processing.  Actual coding starts with the data lake definition.

---
## Use case:  analyzing and scoring a single jump

As a speed skydiver in training, I want to process a single speed jump and get the results, with the ability to view the jump score, summary, and all data points throughout the jump.

In [ ]:
import pathlib

DATA_LAKE = 'quickstart-example/'
jumpFile = pathlib.Path(DATA_LAKE) / 'R1_09-20-26.CSV'

### Getting data from a single jump

In [ ]:
from ssscoring.calc import getFlySightDataFromCSVFileName

rawData, tag = getFlySightDataFromCSVFileName(jumpFile)
display(rawData.head(5))
display('FlySight data: %s' % tag)

### FlySight vs SSScoring data representation

Track files have a unique general purpose format.  SSScoring transforms the generic FlySight headers and data into dataframes with meaningful metadata and actionable units.

In [ ]:
from ssscoring.calc import convertFlySight2SSScoring
data = convertFlySight2SSScoring(rawData, altitudeDZFt=616) # At Skydive Chicago
display(data.head(5))

The SSScoring columns representation is more "programmer friendly" and uses longer, more descriptive names.  It also calculates the altitude AGL, the ISC speed accuracy value for each data point, and provides SI and Imperial units on the same dataframe.  As a user, feel free to use meters or feet directly from the SSScoring jump data representation.

### Evaluating the jump

While the API provides individual functions for validating the jump, separating the skydive from the ground data, separating the speed jump window from the rest of the skydive, and so on, the easiest way to score a jump is to let SSScoring process the jump and return an object with all a dataset of results-related data.

In [ ]:
from ssscoring.calc import processJump
from ssscoring.datatypes import JumpStatus
import warnings
warnings.filterwarnings('ignore', category=UserWarning) # FNV, conda in Jupyter/Lucyfer issue
jumpResult = processJump(data)

A <a href='https://pr3d4t0r.github.io/SSScoring/ssscoring/datatypes.html#JumpResults' target='_blank'>`JumpResults` object</a> reports everything there is to know about the jump:

- `data` contains the speed skydive data, stripped of all other FlySight data prior to exit or below the `BREAKOFF_ALTITUDE`
- `maxSpeed` is the maximum speed attained during the jump
- `score` is the 3-second window max speed, per ISC/FAI/USPA rules
- `scores` contains all the results for every 3-second window during the speed skydive
- `table` has a results table showing the speed, altitude, other data at 5-second intervals
- `window` is a <a href='https://pr3d4t0r.github.io/SSScoring/ssscoring/datatypes.html#PerformanceWindow' target='_blank'>`PerformanceWindow`</a> object with data elements for the start, end, and validation altitude
- `status` is a <a href='https://pr3d4t0r.github.io/SSScoring/ssscoring/datatypes.html#JumpStatus' target='_blank'>`JumpStatus`</a>  object that reports if a jump was successful or not.  If not, its value describes the issue with the current jump.

In [ ]:
if jumpResult.status == JumpStatus.OK:
    display('Score = %.2f, max speed = %.2f' % (jumpResult.score, jumpResult.maxSpeed))
    display(jumpResult.table)
    display(jumpResult.window)

### Output representation

The output in this example is a mishmash of strings.  If using Lucyfer/Jupyter notebooks, the cleanest way to display tabular data is to use dataframes and the built-in `display()` function.  Lucyfer/Jupyter details are outside the scope of this quickstart guide, but here's an example based on the `jumpResult` we have so far:

In [ ]:
import pandas as pd
display(pd.DataFrame({ 'Score': [ round(jumpResult.score, 2), ], 'maxSpeed': [ round(jumpResult.maxSpeed, 2), ] }))
display(jumpResult.table)
display(pd.DataFrame([ jumpResult.window._asdict() ]))

---
## Intepreting the results

SSScoring converts and interprets most speed jump data (new requirements appear every day).  Based on recommendations from many top athletes, one of the most common ways to track progress is to check the performance data on a 1- to 5-second intervals from exit to the end of the speed skydive.  That's how the `table` object came about in the SSScoring `JumpResults`.

In [ ]:
display(jumpResult.table)

Most of the data points are self-explanatory.  Here's how to interpret the other fields:

- `speedAngle` is the skydiver's glide angle with respect to the ground
- `distanceFromExit` is the absolute distance traveled from exit, across the ground; it uses the `lat`, `lon` from the FlySight data and the Haversine formula for calculating the distance between two points on the surface of a sphere
- `netVectorKMh` is the resultant of v-speed and h-speed at every point of the skydive, as reported by the FlySight device, **not** using the ISC/USPA/FAI 3-second window speed calculation

### Jump interpretation

This jumper had a slow score because the angle of attack to the ground was very shallow.  The jumper traveled almost a whole km from exit across the Earth because they were in a steep track.  The data shows that in future jumps it might be a good idea to dive steeper between 5 and 10 seconds, and maintain an angle of at least 80º.  The horizontal speed should trend to the low double- to single-digits (in km/h) toward the end of the skydive.  The net vector and the v-vector should be close in the last 5-10 seconds of the skydive.

---
## Plotting the results

Plots simplify data visualization.  This quickstart uses the Bokeh framework for speed jump visualization.  SSScoring implements a few convenience functions for handling plot initialization.

In [ ]:
from ssscoring.notebook import SPEED_COLORS
from ssscoring.notebook import graphAltitude
from ssscoring.notebook import graphAngle
from ssscoring.notebook import graphJumpResult
from ssscoring.notebook import initializeExtraYRanges
from ssscoring.notebook import initializePlot
import bokeh.models as bm

plot = initializePlot(jumpFile.as_posix())
plot = initializeExtraYRanges(plot, startY=min(jumpResult.data.altitudeAGLFt)-500.0, endY=max(jumpResult.data.altitudeAGLFt)+500.0)
graphAltitude(plot, jumpResult)
graphAngle(plot, jumpResult)
hoverValue = bm.HoverTool(tooltips=[('Y-val', '@y{0.00}',),])
plot.add_tools(hoverValue)
graphJumpResult(plot, jumpResult, lineColor=SPEED_COLORS[0])